# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In /opt/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /opt/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /opt/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
df = pd.read_csv('/Users/felipe/Documents/Springboard/Guided_capstone_project/data/step3_output.csv')
df.drop(columns = 'Unnamed: 0',inplace = True)
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.000000,45.0,44.0,350.0,47.0,53.00000,90.0,0.0,2
1,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.000000,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,1
2,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.000000,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,0
3,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,0,1,...,88.0,92.412903,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,0
4,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,1,3,...,380.0,200.000000,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [3]:
# The aim of creating dummy features is the replacement of categories for numbers, so ML algorithims can handle this data
# I must store the feature or features of interests as a dataframe
dfo = df[['state']] 
pd.get_dummies(dfo) # Using the entries of the categorical feature, get_dummies() will produce a dataframe with the:
# categories as colnames and values either 0 or 1 as entries

# Finally, we removed the feature state and concatenate df(without state feature) with pd.get_dummies(dfo)
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [4]:
# 2nd part: scaling features
# adultweekend = y variable (response for scaling and modeling ?)
# development dataframe ? 

# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
# What is an explanatory variable ?
#  Also known as the independent or predictor variable, it explains variations in the response variable; 
# in an experimental study, it is manipulated by the researcher.

X = df.drop(['Name','AdultWeekend'], axis=1) # we will create a new df called X, without Name and AdultWeekend
# Name is dropped because is not needed. AdultWeekend is our response variable. Why are we removing it ? 

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X) # We will scale all the features, except AdultWeekend

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X

X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [5]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# We will model what we have defined as the response variable(AdultWeekend)
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)
# I dont understand why all the numerical features were scaled, except y (the response variable)
# Maybe we excluded y (response variable) from df, because we want to investigate whether we can predict the values for
# AdultWeekend ?

In [15]:
X_train.shape
y_train.shape

(134,)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [16]:
#all first model set

from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error

lm = linear_model.LinearRegression()

# As we can see, we are building a multiple regression model using all the remaining features as explanatory variables.
# Can we attempt to identify which possess the higher variance ? 

model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [17]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

In [22]:
print(y_pred[:5])
print(y_test[:5])

[56.03448105 90.92071152 69.7180748  76.20000839 34.96368027]
[55. 93. 76. 92. 39.]


## Review Model Outcomes — Iterate over additional models as needed

In [0]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.

# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [19]:
print(explained_variance_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
# This results in single values
# How should I interpret these metrics ?. In other words, how the use of these metrics will tell me whether my predictive
# model is accurate or not ?

-2.7080049704843865e+21
115180125080.42625


**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [24]:
lm.intercept_
# The intercept represents the mean AdultWeekend price for all the resorts. Why is it negative ? 
# Addition or substraction of each coeff values in the regression, results in the predicted AdultWeekend value

# Every feature have an scaled coeff. What is it mean ?. We can determine feature importance

# In the next step, we must:
# print the coefficient values from the linear model
# sort in descending order to the identify the most important features

# Conclusion. A MLR can be helpful to identify the most relevant features to predict the value of a response variable.

-28955897308.747257

In [26]:
# The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. 

# The addition or subtraction of each of the coefficient values in the regression are numeric adjustments 
# applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. 
# Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients 
# for the features to determine the feature importances. 

#Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.

# Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative
# but what we are looking for is the magnitude of impact on our response variable.

In [32]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so:

# The coefficients for every feature are stored in the .coef_ attribute of the linear model
# X.columns, corresponds to the features of the model. Here, we will use them as row-indexes. Additionally, this dataframe
# will contain only one column, named Coefficient.

feat_coefs = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
feat_coefs.sort_values(by='Coefficient',ascending=False)[:10]

# Our model suggest, that the feature with the highest importance is fastEight
#print(feat_coefs.index)

# I do not see that the top ten important features are different states. Only one feature, at 10, correspond to
# the state_New York

,Coefficient
fastEight,2.770224e+13
total_chairs,1.869837e+13
double,1.019004e+13
surface,9.819305e+12
triple,8.347778e+12
fastSixes,6.681448e+12
quad,4.750358e+12
fastQuads,3.940313e+12
trams,3.873346e+12
state_New York,1.632866e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [50]:
# 1 - Excluding the state columns from the dataframe, so we can build a new model
X2 = df.iloc[:,1:25].copy()
X2.drop(['AdultWeekend'], axis=1,inplace = True)
X2.columns
# This dataframe excluded the state columns. Y (Adult weekend price), remains the same

Index(['summit_elev', 'vertical_drop', 'trams', 'fastEight', 'fastSixes',
       'fastQuads', 'quad', 'triple', 'double', 'surface', 'total_chairs',
       'Runs', 'TerrainParks', 'LongestRun_mi', 'SkiableTerrain_ac',
       'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen', 'averageSnowfall',
       'AdultWeekday', 'projectedDaysOpen', 'NightSkiing_ac', 'clusters'],
      dtype='object')

In [51]:
# 2 - Scale the features

scaler = preprocessing.StandardScaler().fit(X2)
X_scaled2 = scaler.transform(X2)

# 3 - Splitting the data into test and train arrays
# As we said, y does not change
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_scaled2, y, test_size=0.25, random_state=1)

# 4 - Building a new linear model

lm2 = linear_model.LinearRegression()
model2 = lm2.fit(X_train2,y_train2)

# 5 - Make a prediction
y_pred2 = model2.predict(X_test2)

# 6 - Identifying the most important features from the new model

feat_coefs2 = pd.DataFrame(abs(lm2.coef_), X2.columns, columns=['Coefficient'])
feat_coefs2.sort_values(by='Coefficient',ascending=False)[:10]

# The results of my model are remarkably different to what I should have according the description

,Coefficient
AdultWeekday,10.922795
clusters,2.807803
daysOpenLastYear,2.350951
vertical_drop,1.779810
TerrainParks,1.636107
averageSnowfall,1.558264
quad,1.527867
double,1.429369
surface,1.163161
projectedDaysOpen,1.059869


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [58]:
# Remove summit_elev. base_elev was already removed during EDA

X3 = X2.drop(['summit_elev'], axis=1).copy()
X3.columns

Index(['vertical_drop', 'trams', 'fastEight', 'fastSixes', 'fastQuads', 'quad',
       'triple', 'double', 'surface', 'total_chairs', 'Runs', 'TerrainParks',
       'LongestRun_mi', 'SkiableTerrain_ac', 'Snow Making_ac',
       'daysOpenLastYear', 'yearsOpen', 'averageSnowfall', 'AdultWeekday',
       'projectedDaysOpen', 'NightSkiing_ac', 'clusters'],
      dtype='object')

In [59]:
# Creating a model without the summit_elev, base_elev and states features

scaler = preprocessing.StandardScaler().fit(X3)
X_scaled3 = scaler.transform(X3)

# 3 - Splitting the data into test and train arrays
# As we said, y does not change
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_scaled3, y, test_size=0.25, random_state=1)

# 4 - Building a new linear model

lm3 = linear_model.LinearRegression()
model3 = lm3.fit(X_train3,y_train3)

# 5 - Make a prediction
y_pred3 = model3.predict(X_test3)

# 6 - Identifying the most important features from the new model

feat_coefs3 = pd.DataFrame(abs(lm3.coef_), X3.columns, columns=['Coefficient'])
feat_coefs3.sort_values(by='Coefficient',ascending=False)[:10]

,Coefficient
AdultWeekday,11.026285
clusters,2.505131
daysOpenLastYear,2.399476
vertical_drop,1.889586
TerrainParks,1.657870
quad,1.604710
averageSnowfall,1.405882
double,1.397772
surface,1.248481
projectedDaysOpen,1.088222


In [70]:
## Identify the Final Model
# Create a dataframe resuming the findings

lst_model1 = [explained_variance_score(y_test,y_pred),mean_absolute_error(y_test,y_pred)]
lst_model2 = [explained_variance_score(y_test2,y_pred2),mean_absolute_error(y_test2,y_pred2)]
lst_model3 = [explained_variance_score(y_test3,y_pred3),mean_absolute_error(y_test3,y_pred3)]

metrics_summary = pd.DataFrame([lst_model1,lst_model2,lst_model3],
                               index=['Model1', 'Model2', 'Model3'],
                               columns = ['variance_score','mean_error'])

metrics_summary['Metadata'] = ['No features removed', 'States features removed',['States features removed','summit_elev']]
metrics_summary

,variance_score,mean_error,Metadata
Model1,-2.708005e+21,1.151801e+11,No features removed
Model2,6.461586e-01,6.724178e+00,States features removed
Model3,6.431679e-01,6.735532e+00,"[States features removed, summit_elev]"


**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

Model Selection:

In [71]:
# I select model number 3.